In [ ]:
from jupyter_dash import JupyterDash 

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import pandas as pd

# Import your CRUD Python module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Update these with your credentials
username = "aacuser"
password = "SNHU1234"

# Connect to the database via CRUD Module
db = AnimalShelter(username, password)

# Fetch data and drop MongoDB `_id` column
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)


image_filename = 'Grazioso Salvare Logo.png'  # Replace with your image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),  # Hidden div for debug
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),  # Dashboard title
    html.Center(html.H2("Mohamed Aziz Zaghdoudi")),  # Unique identifier
        html.Center(html.A(
        href="https://www.snhu.edu",  # Anchor tag URL to the client's homepage
        children=html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()), 
            style={'height': '10%', 'width': '10%'}
        ),
        target="_blank"  # Opens the link in a new tab
    )),
    html.Hr(),

    # Interactive Filter Options
    html.Div([
        html.Label('Interactive Filter Options:'),
        dcc.RadioItems(
            id='filter-options',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            inline=True
        )
    ], style={'margin-bottom': '20px'}),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable='single',
        selected_rows=[0],
        style_data_conditional=[
            {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}
        ],
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        }
    ),
    html.Br(),

    # Charts
    html.Div([
        dcc.Graph(id='pie-chart'),
        html.Div(id='map-id', className='col s12 m6')
    ], style={'display': 'flex', 'justify-content': 'space-between'}),
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-options', 'value')]
)
# Updating data for the selected filter 
def update_data(selected_filter):
    """Filter the DataTable based on the selected rescue type."""
    if selected_filter == 'Reset':
        filtered_df = df
    elif selected_filter == 'Water Rescue':
        filtered_df = df[
            (df['breed'].str.contains('Labrador Retriever|Chesapeake Bay Retriever|Newfoundland', case=False, na=False)) &
            (df['sex_upon_outcome'].str.contains('Intact Female', case=False, na=False)) &
            (df['age_upon_outcome_in_weeks'].between(26, 156))
        ]
    elif selected_filter == 'Mountain or Wilderness Rescue':
        filtered_df = df[
            (df['breed'].str.contains('German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler', case=False, na=False)) &
            (df['sex_upon_outcome'].str.contains('Intact Male', case=False, na=False)) &
            (df['age_upon_outcome_in_weeks'].between(26, 156))
        ]
    elif selected_filter == 'Disaster Rescue or Individual Tracking':
        filtered_df = df[
            (df['breed'].str.contains('Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler', case=False, na=False)) &
            (df['sex_upon_outcome'].str.contains('Intact Male', case=False, na=False)) &
            (df['age_upon_outcome_in_weeks'].between(20, 300))
        ]
    else:
        filtered_df = df

    return filtered_df.to_dict('records')


@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
#Pie chart function 
def update_pie_chart(table_data):
    """Update the pie chart based on the filtered data."""
    if table_data is None or len(table_data) == 0:
        return px.pie(title='No Data Available')

    dff = pd.DataFrame(table_data)
    pie_data = dff['breed'].value_counts().reset_index()
    pie_data.columns = ['breed', 'count']

    top_n = 10
    pie_data['breed'] = pie_data.apply(
        lambda row: row['breed'] if row.name < top_n else 'Others', axis=1
    )
    pie_data = pie_data.groupby('breed', as_index=False).sum()

    return px.pie(
        pie_data,
        names='breed',
        values='count',
        title=f'Top {top_n} Breeds Distribution'
    )


@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
#Updating the map for each selected Row 
def update_map(viewData, index):
    """Update the geolocation map based on the selected row."""
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available")]

    dff = pd.DataFrame.from_dict(viewData)

    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    try:
        latitude = dff.iloc[row]['location_lat']
        longitude = dff.iloc[row]['location_long']
        breed = dff.iloc[row]['breed']
        animal_name = dff.iloc[row]['name']

        return [
            dl.Map(style={'width': '500px', 'height': '500px'},
                   center=[latitude, longitude], zoom=10, children=[
                       dl.TileLayer(id="base-layer-id"),
                       dl.Marker(position=[latitude, longitude], children=[
                           dl.Tooltip(breed),
                           dl.Popup([html.H1("Animal Name"), html.P(animal_name)])
                       ])
                   ])
        ]
    except Exception as e:
        print(f"Error occurred: {e}")
        return [html.P("Unable to load map data.")]


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


ModuleNotFoundError: No module named 'jupyter_dash'